# RAG Analysis for healthcare challenges

### Business Context
The healthcare industry is rapidly evolving, and professionals face increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. Quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.
Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care. 
To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.
### Objective
As an Al specialist, your task is to develop a RAG-based Al solution using renowned medical manuals to address healthcare challenges. The objective is to understand information overload, apply Al techniques to streamline decision-making, analyze its impact on diagnostics and patient outcomes, evaluate its potential to standardize care practices, and create a functional prototype demonstrating its feasibility and effectiveness.

####  Questions to Answer
1. What is the protocol for managing sepsis in a critical care unit?
2. What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
3. What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
4. What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?
5. What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?

### downloading required libraries

In [1]:
!pip install langchain transformers chromadb sentence-transformers ollama pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.0 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 77.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2

### Defining Classes

In [5]:
!pip install -U langchain-community

from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import ollama

# --- Data Processing Classes ---
class DataLoader:
    def __init__(self, file_path):
        self.file_path = file_path

    def load_data(self):
        if self.file_path.lower().endswith(".pdf"):
            loader = PyPDFLoader(self.file_path)
        elif self.file_path.lower().endswith(".txt"):
            loader = TextLoader(self.file_path)
        else:
            raise ValueError("Unsupported file type. Please provide a .pdf or .txt file.")
        return loader.load()

class DataSplitter:
    def __init__(self, chunk_size=500, chunk_overlap=50):
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=self.chunk_size, chunk_overlap=self.chunk_overlap
        )

    def split_data(self, documents):
        return self.text_splitter.split_documents(documents)

# --- Embedding Class ---
class EmbeddingModel:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.embeddings = HuggingFaceEmbeddings(model_name=self.model_name)

    def get_embeddings(self, texts):
        return self.embeddings.embed_documents(texts)

    def embed_query(self, query):
        return self.embeddings.embed_query(query)

# --- Vector Database Class ---
from langchain_community.vectorstores import Chroma

class VectorDB:
    def __init__(self, persist_directory="chroma_db", embedding_function=None):
        self.persist_directory = persist_directory
        self.embedding_function = embedding_function
        self.vectordb = None

    def create_and_store(self, documents):
        if self.embedding_function is None:
            raise ValueError("Embedding function must be provided.")
        self.vectordb = Chroma.from_documents(
            documents=documents,
            embedding=self.embedding_function,
            persist_directory=self.persist_directory
        )
        self.vectordb.persist()
        print(f"Vector database created and persisted in: {self.persist_directory}")

    def load_existing(self):
        if self.embedding_function is None:
            raise ValueError("Embedding function must be provided.")
        self.vectordb = Chroma(
            persist_directory=self.persist_directory,
            embedding_function=self.embedding_function
        )
        print(f"Loaded existing vector database from: {self.persist_directory}")

    def get_retriever(self, search_type="similarity", search_kwargs=None):
        if self.vectordb is None:
            raise ValueError("Vector database not initialized. Call create_and_store or load_existing first.")
        if search_kwargs is None:
            search_kwargs = {"k": 3}
        return self.vectordb.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

# --- LLM Interaction Class ---
class LocalLLM:
    def __init__(self, model_name="llama3", default_max_length=512, default_temperature=0.7):
        self.model_name = model_name
        self.default_max_length = default_max_length
        self.default_temperature = default_temperature

    def generate_response(self, prompt, max_length=None, temperature=None, system_message=None):
        messages = []
        if system_message:
            messages.append({'role': 'system', 'content': system_message})
        messages.append({'role': 'user', 'content': prompt})

        try:
            response = ollama.chat(
                model=self.model_name,
                messages=messages,
                options={
                    'num_predict': max_length if max_length is not None else self.default_max_length,
                    'temperature': temperature if temperature is not None else self.default_temperature,
                }
            )
            return response['message']['content']
        except ollama.OllamaAPIError as e:
            return f"Error communicating with Ollama: {e}"

### initializing paths and questions 

In [13]:
data_file_path = "/kaggle/input/med-manual/med_manual.pdf"  # Adjust path if needed
persist_directory = "chroma_db"  # Will be created in /kaggle/working/
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
llm_model_name = "llama3"  # Ensure Ollama is running with this model
questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]

### Data Preprocessing

In [14]:
# Load Data
data_loader = DataLoader(data_file_path)
documents = data_loader.load_data()

# Split Data
data_splitter = DataSplitter()
chunks = data_splitter.split_data(documents)

# Embed and Store in Vector Database
embedding_model = EmbeddingModel(model_name=embedding_model_name)
vector_db = VectorDB(persist_directory=persist_directory, embedding_function=embedding_model.embeddings)
try:
    vector_db.load_existing()
    print("Loaded existing vector database.")
except:
    vector_db.create_and_store(chunks)
    print("Created and stored new vector database.")

# Define Retriever
retriever = vector_db.get_retriever(search_type="similarity", search_kwargs={"k": 3})

# Load LLM
llm = LocalLLM(model_name=llm_model_name)

<ipython-input-5-c4b515652cbe>:38: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  self.embeddings = HuggingFaceEmbeddings(model_name=self.model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

<ipython-input-5-c4b515652cbe>:69: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  self.vectordb = Chroma(


Loaded existing vector database from: chroma_db
Loaded existing vector database.


### Answering questions using (llm) with out RAG

In [17]:
from transformers import pipeline

class HFLLM:
    def __init__(self, model_name="google/flan-t5-large"): # Choose a suitable model
        self.model_name = model_name
        self.generator = pipeline("text2text-generation", model=self.model_name)

    def generate_response(self, prompt, max_length=512, temperature=0.7, system_message=None):
        # System message might need to be incorporated into the prompt differently
        response = self.generator(prompt, max_length=max_length, temperature=temperature, num_return_sequences=1, do_sample=True) # Added do_sample for temperature to have effect
        return response[0]['generated_text']

In [18]:
# --- 3. Load LLM ---
llm = HFLLM(model_name="google/flan-t5-large") # You can try other models like "facebook/bart-large-cnn" or similar

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [19]:
print("\n--- Question Answering using LLM (Without RAG) ---")
questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]
for question in questions:
    prompt = f"Answer the following medical question directly in 2-3 sentences: {question}"
    response = llm.generate_response(prompt)
    print(f"\nQuestion: {question}")
    print(f"Answer:\n{response}")
    print("-" * 50)


--- Question Answering using LLM (Without RAG) ---

Question: What is the protocol for managing sepsis in a critical care unit?
Answer:
Sepsis in critical care unit should be managed on a case-by-case basis.
--------------------------------------------------

Question: What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
Answer:
Symptoms of appendicitis are pain , burning, and a burning sensation in the belly area. The pain can be felt right down your thigh and may radiate to your groin or other parts of your abdomen. The pain can be exacerbated by a viral infection or by a condition called appendicitis.
--------------------------------------------------

Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
Answer:
dietary changes
----------------------

## observations:

Based on these observations, the LLM, without access to specific medical knowledge provided through RAG, struggles to provide accurate, complete, and helpful answers to medical questions. The responses are often too general, lack specific details, and can even be misleading or factually questionable. This highlights the need for grounding the LLM's knowledge with relevant external information when dealing with specialized domains like medicine.

These observations strongly support the rationale for using Retrieval-Augmented Generation (RAG) to provide the LLM with the necessary context to answer these questions more effectively.

### Question Answering using LLM with Prompt Engineering and fine tuning

In [20]:
print("\n--- Question Answering using LLM with Prompt Engineering (5+ Combinations) ---")

prompt_tuning_combinations = [
    {
        "prompt": "As a knowledgeable medical professional, please provide a concise answer (2-3 sentences) to the question: {}",
        "temperature": 0.3,
        "comment": "Concise expert answer, lower temperature for more focused output."
    },
    {
        "prompt": "Explain in detail (4-5 sentences) the answer to the following medical query: {}",
        "temperature": 0.7,
        "comment": "Detailed explanation, moderate temperature for balanced creativity."
    },
    {
        "prompt": "Imagine you are teaching a medical student. Explain the concept related to the question: {} in simple terms (3-4 sentences).",
        "temperature": 0.5,
        "comment": "Simplified explanation for teaching, moderate temperature."
    },
    {
        "prompt": "What are the key aspects and recommendations for the medical situation described in this question: {}? Provide a brief summary (2 sentences).",
        "temperature": 0.4,
        "comment": "Key aspects and brief summary, slightly lower temperature for focus."
    },
    {
        "prompt": "Think step-by-step and provide a logical answer (4 sentences) to the medical question: {}",
        "temperature": 0.6,
        "comment": "Step-by-step logical answer, moderate temperature."
    },
    {
        "prompt": "Be creative and provide a potential answer (3 sentences) to the medical question: {}",
        "temperature": 0.9,
        "comment": "Creative potential answer, higher temperature for more variability."
    },
]

questions = [
    "What is the protocol for managing sepsis in a critical care unit?",
    "What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?",
    "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?",
    "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?",
    "What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?"
]

for i, combo in enumerate(prompt_tuning_combinations):
    print(f"\n--- Combination {i+1}: {combo['comment']} (Temperature: {combo['temperature']}) ---")
    prompt_template = combo["prompt"]
    temperature = combo["temperature"]

    for j, question in enumerate(questions):
        engineered_prompt = prompt_template.format(question)
        response = llm.generate_response(engineered_prompt, temperature=temperature)
        print(f"\nQuestion: {question}")
        print(f"Answer:\n{response}")
        print("-" * 50)


--- Question Answering using LLM with Prompt Engineering (5+ Combinations) ---

--- Combination 1: Concise expert answer, lower temperature for more focused output. (Temperature: 0.3) ---

Question: What is the protocol for managing sepsis in a critical care unit?
Answer:
The protocol for managing sepsis in a critical care unit is:
--------------------------------------------------

Question: What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
Answer:
Common symptoms of appendicitis are : Painful swollen appendages, a swollen urethra, and a swollen urethra. It can be cured via medicine, but surgery is the only option.
--------------------------------------------------

Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
Answer:
Medications for hair lo

### Observations: Question Answering using LLM with Prompt Engineering (5+ Combinations)

**Overall:** The quality of answers varied greatly based on the prompt and temperature settings. While some combinations showed slight improvements over the "without RAG" results, a lack of specific medical knowledge still hampered accuracy and completeness.

**Impact of Temperature:**

* **Lower Temperatures (0.3-0.4):** Resulted in more concise and focused responses, but often lacked detail or were overly generic.
* **Moderate Temperatures (0.5-0.7):** Led to more elaborate and creative attempts but also increased the risk of inaccuracies and irrelevant information.
* **Higher Temperature (0.9):** Produced the most varied and creative answers but also the highest frequency of nonsensical or factually incorrect content.

**Impact of Prompt Engineering:**

Engineered prompts provided some guidance on the style and length of the responses. However, without access to relevant medical data, the prompts alone could not ensure accuracy or depth of knowledge.

**Specific Combination Observations:**

* **Combination 1 (Concise expert, Temp 0.3):** Answers were very brief, often incomplete sentences or single, potentially insufficient pieces of information.
* **Combination 2 (Detailed explanation, Temp 0.7):** Attempts at detailed explanations included factual errors and vague information. Advice provided could also be misleading.
* **Combination 3 (Simplified explanation, Temp 0.5):** Simplified explanations often contained inaccuracies and misrepresentations of medical concepts.
* **Combination 4 (Key aspects, brief summary, Temp 0.4):** Frequently resulted in repetitions of the question or very general statements without specific details.
* **Combination 5 (Step-by-step logical, Temp 0.6):** While aiming for a logical flow, answers still suffered from factual errors and incomplete information.
* **Combination 6 (Creative potential, Temp 0.9):** Demonstrated the most variability but also the highest likelihood of incorrect or nonsensical answers.

**Conclusion:**

Prompt engineering and temperature tuning alone are insufficient to generate reliable and accurate medical answers from the base LLM. The absence of specific medical knowledge remains a significant limitation. These results underscore the necessity of integrating relevant information through techniques like Retrieval-Augmented Generation (RAG) to improve the quality of responses in specialized domains.

###  Question Answering using RAG (Initial Run)

In [21]:
print("\n--- Question Answering using RAG (Initial Run) ---")
for question in questions:
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    prompt = f"Based on the following medical context, answer the question: '{question}'\n\nContext:\n{context}\n\nAnswer:"
    response = llm.generate_response(prompt)
    print(f"\nQuestion: {question}")
    print(f"Answer:\n{response}")
    print("-" * 50)


--- Question Answering using RAG (Initial Run) ---


<ipython-input-21-3f3557390588>:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)



Question: What is the protocol for managing sepsis in a critical care unit?
Answer:
In the intensive care unit, sepsis is treated as a non life threatening emergency condition.
--------------------------------------------------

Question: What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?
Answer:
abdominal pain
--------------------------------------------------

Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?
Answer:
Medications
--------------------------------------------------

Question: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?
Answer:
psychiatric interventions
--------------------------------------------------

Question: W

### Observations: Question Answering using RAG (Initial Run)

**Overall:** The initial RAG results show some improvement over the standalone LLM, as the answers are now grounded in retrieved context. However, the answers are still generally brief and sometimes lack sufficient detail or accuracy.

**Specific Observations for Each Question:**

* **Question 1: What is the protocol for managing sepsis in a critical care unit?**
    * **Answer:** "In the intensive care unit, sepsis is treated as a non life threatening emergency condition."
    * **Observation:** This is inaccurate. Sepsis in a critical care unit is a life-threatening emergency requiring immediate and aggressive intervention. The retrieved context seems to have led to a misunderstanding.

* **Question 2: What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?**
    * **Answer:** "abdominal pain"
    * **Observation:** While abdominal pain is a key symptom of appendicitis, the answer is incomplete. It doesn't address other common symptoms, whether it's curable by medicine (it generally isn't), or the surgical procedure (appendectomy).

* **Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?**
    * **Answer:** "Medications"
    * **Observation:** This is a very general answer. While medications are used to treat some causes of patchy hair loss, the response lacks specifics on the types of medications or potential causes.

* **Question 4: What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?**
    * **Answer:** "psychiatric interventions"
    * **Observation:** While psychiatric support can be important for patients with brain injuries, it's not the primary treatment for the physical injury itself. This answer misses crucial aspects like immediate medical care, rehabilitation, and other therapies.

* **Question 5: What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery?**
    * **Answer:** "The following are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery: Ice packs are used to treat fractures caused by sudden and unrelenting movements, such as walking, climbing, or running."
    * **Observation:** The answer starts by acknowledging the question but then only mentions ice packs as a treatment. While ice is helpful for swelling and pain, it's a very incomplete answer regarding the necessary precautions (immobilization, seeking medical help) and the full spectrum of treatment and recovery.

**Conclusion:**

The initial RAG run demonstrates the potential to ground answers in provided data, but the results are still quite basic and sometimes inaccurate. The retrieved context might not always be the most relevant or comprehensive, and the LLM might not be effectively synthesizing the information into complete and accurate answers. Fine-tuning the retriever and LLM parameters in the subsequent steps will be crucial to improve the performance of the RAG system.

In [22]:
print("\n--- Fine-tuning Retriever and LLM Parameters for RAG ---")
rag_tuning_combinations = [
    {"retriever_k": 2, "llm_temp": 0.6, "prompt_style": "concise"},
    {"retriever_k": 4, "llm_temp": 0.8, "prompt_style": "detailed"},
    {"retriever_k": 3, "llm_temp": 0.4, "prompt_style": "step-by-step"},
    {"retriever_k": 5, "llm_temp": 0.7, "prompt_style": "expert"},
    {"retriever_k": 2, "llm_temp": 0.9, "prompt_style": "summary"},
]

for i, combo in enumerate(rag_tuning_combinations):
    print(f"\n--- RAG Combination {i+1} ---")
    retriever_tuned = vector_db.get_retriever(search_kwargs={"k": combo["retriever_k"]})
    llm_temp = combo["llm_temp"]
    prompt_style = combo["prompt_style"]

    for question in questions:
        relevant_docs = retriever_tuned.get_relevant_documents(question)
        context = "\n".join([doc.page_content for doc in relevant_docs])

        if prompt_style == "concise":
            prompt = f"Based on the context, concisely answer: '{question}'\n\nContext:\n{context}\n\nAnswer:"
        elif prompt_style == "detailed":
            prompt = f"Based on the following medical context, provide a detailed answer to: '{question}'\n\nContext:\n{context}\n\nAnswer:"
        elif prompt_style == "step-by-step":
            prompt = f"Based on the context, explain the steps involved in addressing: '{question}'\n\nContext:\n{context}\n\nAnswer:"
        elif prompt_style == "expert":
            prompt = f"As a medical expert, based on the provided context, what is your assessment and recommendation for '{question}'?\n\nContext:\n{context}\n\nAnswer:"
        elif prompt_style == "summary":
            prompt = f"Based on the context, summarize the key information related to: '{question}' in a few sentences.\n\nContext:\n{context}\n\nAnswer:"
        else:
            prompt = f"Based on the following medical context, answer the question: '{question}'\n\nContext:\n{context}\n\nAnswer:"

        response = llm.generate_response(prompt, temperature=llm_temp)
        print(f"\nQuestion: {question} (k={combo['retriever_k']}, temp={llm_temp}, style='{prompt_style}')")
        print(f"Answer:\n{response}")
        print("-" * 50)


--- Fine-tuning Retriever and LLM Parameters for RAG ---

--- RAG Combination 1 ---

Question: What is the protocol for managing sepsis in a critical care unit? (k=2, temp=0.6, style='concise')
Answer:
Patient care instructions
--------------------------------------------------

Question: What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? (k=2, temp=0.6, style='concise')
Answer:
Appendicitis is the most common cause of sudden loss of a fecal sac.
--------------------------------------------------

Question: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? (k=2, temp=0.6, style='concise')
Answer:
Hair loss can be caused by a variety of causes, including hormones, cancer, autoimmune disease, and hormonal imbalances.
-------------------------------------------

# Manual Evaluation of RAG Responses

This document presents a manual evaluation of the Retrieval-Augmented Generation (RAG) system's output for various medical questions across different parameter configurations. The groundedness assessment is based on general medical knowledge due to the absence of the specific retrieval context.

## Evaluation Results by RAG Combination

---

### RAG Combination 1 (k=2, temp=0.6, style='concise')

| Question                                                                                                                                  | Answer                                                                                                                                  | Groundedness Score (Justification)                                                                                                                                                                                                                                                           | Relevance Score (Justification)                                                                                                                                                                                                                                                              |
| :---------------------------------------------------------------------------------------------------------------------------------------- | :---------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| What is the protocol for managing sepsis in a critical care unit?                                                                         | Patient care instructions                                                                                                                 | 2/5 (Too vague; lacks specifics of a sepsis protocol.)                                                                                                                                                                                                                                         | 2/5 (Broadly related to patient management, but doesn't address the sepsis protocol.)                                                                                                                                                                                                   |
| What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? | Appendicitis is the most common cause of sudden loss of a fecal sac.                                                              | 1/5 (Factually incorrect; appendicitis involves appendix inflammation, not loss of a fecal sac.)                                                                                                                                                                                                   | 1/5 (Fails to address symptoms, medical curability, or surgical procedure.)                                                                                                                                                                                                                 |
| What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? | Hair loss can be caused by a variety of causes, including hormones, cancer, autoimmune disease, and hormonal imbalances.                                                              | 4/5 (Generally accurate causes for various hair loss types.)                                                                                                                                                                                                                                 | 3/5 (Mentions causes, but omits discussion of treatments for patchy hair loss.)                                                                                                                                                                                                           |
| What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function? | acupuncture                                                                                                             | 1/5 (Not a primary or widely recommended treatment for direct brain tissue injury.)                                                                                                                                                                                                     | 2/5 (Suggests a treatment, but not central to addressing brain injury.)                                                                                                                                                                                                                   |
| What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery? | Taking proper precautions and avoiding Falls                                                                                              | 3/5 (Preventing falls is a general precaution, but doesn't cover treatment of an existing fracture.)                                                                                                                                                                                          | 2/5 (Mentions precautions, but misses treatment and recovery for a fractured leg.)                                                                                                                                                                                                        |

---

### RAG Combination 2 (k=4, temp=0.8, style='detailed')

| Question                                                                                                                                  | Answer                                                                                                                                                                 | Groundedness Score (Justification)                                                                                                                                                                                                                                                           | Relevance Score (Justification)                                                                                                                                                                                                                                                              |
| :---------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| What is the protocol for managing sepsis in a critical care unit?                                                                         | The Intensive Care Unit of the Medical Center of the City of London                                                                                    | 1/5 (A location, not a medical protocol.)                                                                                                                                                                                                                                               | 1/5 (Does not address the question about a medical protocol.)                                                                                                                                                                                                                                   |
| What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? | Acute appendicitis is usually caused by a bacterial infection or an infection of the digestive tract . It is treated by surgically removing the appendicitis. | 4/5 (Accurate regarding cause and primary treatment.)                                                                                                                                                                                                                                 | 3/5 (Mentions cause and surgical treatment, but omits symptoms and medical curability.)                                                                                                                                                                                                        |
| What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? | Hair loss can be caused by a wide variety of causes, including genetics, environmental factors, and a variety of environmental conditions.                                                              | 4/5 (Generally recognized categories of hair loss causes.)                                                                                                                                                                                                                                 | 3/5 (Addresses causes but not specific treatments for patchy hair loss.)                                                                                                                                                                                                           |
| What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function? | a combination of medications and lifestyle changes                                                                                             | 3/5 (General; medications and lifestyle changes are part of management but lacks specifics for acute injury.)                                                                                                                                                                                           | 3/5 (Vaguely mentions treatments but lacks specifics for brain injury.)                                                                                                                                                                                                                   |
| What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery? | Preparation for hiking should be done when hiking. This includes walking slowly, making sure your feet are level with the ground and taking any necessary physiotherapy prescriptions. Do not hike while in a state of shock, and after a period of time in which you have been awoken from sleep by a strong and alarming pain in your leg, do not hike while unzipped or covered in clothing so as to not suffocate. If you think you may be a victim of a hiking injury, seek medical attention immediately. | 3/5 (Includes relevant advice (medical attention, physiotherapy) but also nonsensical/irrelevant statements.)                                                                                                                                                                                      | 3/5 (Touches on precautions, treatment, and care aspects, but irrelevant details detract.)                                                                                                                                                                                                   |

---

### RAG Combination 3 (k=3, temp=0.4, style='step-by-step')

| Question                                                                                                                                  | Answer                                                                                                                                                                                                                                                                                                                                                                                                                      | Groundedness Score (Justification)                                                                                                                                                                                                                                                                                                                                                                                                                      | Relevance Score (Justification)                                                                                                                                                                                                                                                              |
| :---------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| What is the protocol for managing sepsis in a critical care unit?                                                                         | The protocol for managing sepsis in a critical care unit is:                                                                                                                                                                                                                                                                                                                                                      | 1/5 (Restatement of the question; provides no actual protocol information.)                                                                                                                                                                                                                                                                                                                                                                    | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |
| What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? | The common symptoms of appendicitis are: Pain in the lower abdomen, vomiting, and abdominal pain. The pain is usually in the lower abdomen, but it may also be in the lower abdomen. The pain may be caused by a virus, especially a virus that causes the bacterial infection. The bacterial infection may be treated with antibiotics.                                                              | 3/5 (Lists some common symptoms; viral cause less typical; mentions antibiotics but not definitive curability or surgery.)                                                                                                                                                                                                                                                                                                            | 3/5 (Addresses symptoms and touches on medical treatment, but not fully about surgery or medical curability.)                                                                                                                                                                                          |
| What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? | The topic of the article is hair loss.                                                                                                                                                                                                                                                                                                                            | 1/5 (Provides no information about causes or treatments.)                                                                                                                                                                                                                                                                                                                                                                                    | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |
| What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function? | The person has suffered a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function. The person is treated with medications to help restore normal brain function.                                                                                                                                                                                          | 2/5 (Restates the problem; vaguely mentions medications without specifics.)                                                                                                                                                                                                                                                                                                                                                                 | 2/5 (Acknowledges injury and mentions a treatment type, but very general.)                                                                                                                                                                                                                   |
| What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery? | A person who has fractured their leg during a hiking trip should take precautions to avoid amputations, and should consider getting medical attention.                                                                                                                                                                                                                            | 3/5 (Seeking medical attention is crucial; amputation risk possible in severe cases.)                                                                                                                                                                                                                                                                                                                                                           | 3/5 (Mentions a key treatment step and a consideration, but lacks detail on immediate precautions or recovery.)                                                                                                                                                                              |

---

### RAG Combination 4 (k=5, temp=0.7, style='expert')

| Question                                                                                                                                  | Answer                                                                                                                                | Groundedness Score (Justification)                                                                                                                                                                                                                                                           | Relevance Score (Justification)                                                                                                                                                                                                                                                              |
| :---------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------ | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| What is the protocol for managing sepsis in a critical care unit?                                                                         | It is likely that an individual should be woken with a red scream and placed on a gurney.                                               | 1/5 (Nonsensical and inappropriate for sepsis management.)                                                                                                                                                                                                                                  | 1/5 (Does not address the question about a medical protocol.)                                                                                                                                                                                                                                   |
| What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? | Yes                                                                                                                                 | 1/5 (Insufficient and potentially misleading; doesn't address symptoms or surgery.)                                                                                                                                                                                                       | 1/5 (Inadequately addresses only one part of the multi-part question.)                                                                                                                                                                                                                         |
| What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? | a.                                                                                                    | 1/5 (Not a meaningful or informative answer.)                                                                                                                                                                                                                                 | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |
| What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function? | The need for medical care can be mitigated by self-care, physical activity, and avoiding alcohol and caffeine. | 2/5 (Self-care is supportive but not primary treatment for acute brain injury; avoiding substances is sensible but insufficient.)                                                                                                                                                              | 2/5 (Touches on managing the situation but not specific medical treatments.)                                                                                                                                                                                                           |
| What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery? | ii.                                                                                                   | 1/5 (Not a meaningful or informative answer.)                                                                                                                                                                                                                                 | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |

---

### RAG Combination 5 (k=2, temp=0.9, style='summary')

| Question                                                                                                                                  | Answer                                                                                                                                                                                                                                                                                                                                                                                                                      | Groundedness Score (Justification)                                                                                                                                                                                                                                                           | Relevance Score (Justification)                                                                                                                                                                                                                                                              |
| :---------------------------------------------------------------------------------------------------------------------------------------- | :----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | :------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ |
| What is the protocol for managing sepsis in a critical care unit?                                                                         | Identify the patient in order to ensure they receive the appropriate treatment.                                                                                                                                                                                                                                                                                                                                        | 3/5 (Basic but incomplete step in sepsis management.)                                                                                                                                                                                                                                      | 2/5 (Mentions a basic step but lacks protocol details.)                                                                                                                                                                                                                                          |
| What are the common symptoms of appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it? | This article explains what are the common symptoms of appendicitis, and can it be cured via medicine? If not, which surgical procedure should be followed to treat it? . It describes each of these conditions and how to treat these conditions .                                                              | 1/5 (Meta-description, not the actual information.)                                                                                                                                                                                                                                   | 1/5 (Does not provide any of the requested information.)                                                                                                                                                                                                                                   |
| What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it? |  About the author : Richard rew’Kehoe: a.k.a. Svenda rnsson: A.k.a. Svenda to p.                                                                                                                                                                                                                                                                                          | 1/5 (Information about an author, not medically relevant.)                                                                                                                                                                                                                                   | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |
| What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function? | A person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function should receive treatment.                                                                                                                                                                                                                         | 2/5 (Logical but very basic; no specific treatment recommendations.)                                                                                                                                                                                                                         | 2/5 (Acknowledges need for treatment but offers no details.)                                                                                                                                                                                                                   |
| What are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip, and what should be considered for their care and recovery? | The following are the necessary precautions and treatment steps for a person who has fractured their leg during a hiking trip , and what should be considered for their care and recovery .                                                                                                                                                                                          | 1/5 (Placeholder statement; provides no actual information.)                                                                                                                                                                                                                                | 1/5 (Does not address the question.)                                                                                                                                                                                                                                                          |

## Overall Comments on Evaluation Output

The manual evaluation, based on general medical knowledge, reveals a consistent pattern of low groundedness and relevance across all tested RAG combinations. Key observations include:

* **Predominantly Poor Performance:** The majority of answers fail to provide accurate, specific, and relevant information in response to the medical questions.
* **Lack of Medical Accuracy:** Several answers contain factual inaccuracies or nonsensical statements within a medical context.
* **Insufficient Detail:** Even when somewhat relevant, answers often lack the depth and specificity expected for medical inquiries.
* **Meta-Information and Placeholders:** Some answers provide descriptions of potential content or act as placeholders rather than delivering the requested information.
* **Inconclusive Parameter Impact:** Without the retrieval context, it's challenging to determine how the parameters (`k`, `temperature`, `style`) directly influenced the quality of the retrieved information and subsequent answers. However, none of the combinations yielded consistently satisfactory results.

**Recommendations for Improvement:**

1.  **Enhance Retrieval Mechanism:** The RAG system's ability to retrieve pertinent and accurate medical information needs significant improvement. Investigate the knowledge base, indexing strategy, and query formulation.
2.  **Refine Prompt Engineering:** The prompts used to instruct the LLM should be carefully reviewed and optimized to encourage more accurate, detailed, and relevant responses, tailored to the specific style requested.
3.  **Consider LLM Capabilities:** Evaluate whether the current LLM possesses sufficient medical knowledge and reasoning capabilities. Fine-tuning on medical datasets or exploring models specifically designed for healthcare applications might be necessary.
4.  **Address Generation Issues:** The occurrence of nonsensical or placeholder answers suggests potential problems in the generation process that need to be identified and resolved.

A more comprehensive evaluation would require access to the context used for each answer to differentiate between issues arising from retrieval and those stemming from the language model's generation process. However, based on this assessment, substantial improvements are needed across the RAG pipeline to effectively answer medical questions.

### Observations: Fine-tuning Retriever and LLM Parameters for RAG

**Overall:** The results of fine-tuning the retriever's `k` value, the LLM's temperature, and the prompt style show a mixed bag of outcomes. Some combinations provide slightly more relevant snippets, but the LLM still struggles to synthesize comprehensive and accurate answers consistently. Many responses remain very brief, lack specific details, or even introduce inaccuracies.

**Combination 1 (k=2, temp=0.6, style='concise'):**

* Answers are extremely concise, often just a few words or a fragment, lacking any real substance or detail.

**Combination 2 (k=4, temp=0.8, style='detailed'):**

* Attempts at more detailed answers are still often vague or contain inaccuracies (e.g., the cause of appendicitis). The advice for a fractured leg includes irrelevant information about hiking in a state of shock.

**Combination 3 (k=3, temp=0.4, style='step-by-step'):**

* The "step-by-step" prompt often leads to the model restating the question or providing very basic and incomplete steps.

**Combination 4 (k=5, temp=0.7, style='expert'):**

* The "expert" prompt with a moderate temperature yields some of the most nonsensical and unhelpful responses ("woken with a red scream," "Yes," "a.").

**Combination 5 (k=2, temp=0.9, style='summary'):**

* The "summary" prompt at a higher temperature often results in the model summarizing the question itself or providing very generic statements.

**Trends and Insights:**

* **`k` Value:** Changing the number of retrieved documents (`k`) doesn't consistently lead to better answers. A higher `k` might provide more context, but if the LLM struggles to synthesize it, the answer doesn't necessarily improve.
* **Temperature:** Adjusting the temperature influences the creativity and randomness, but without a strong grounding in accurate medical knowledge, higher temperatures can exacerbate inaccuracies. Lower temperatures tend to produce more focused but still often incomplete responses.
* **Prompt Style:** Different prompt styles can guide the format of the answer, but they cannot compensate for a lack of relevant information or the LLM's inability to reason effectively about the medical context.

**Conclusion:**

These fine-tuning experiments highlight the challenges of relying solely on a general-purpose LLM, even with RAG, for complex medical questions. While the RAG system retrieves some relevant information, the LLM's ability to understand, synthesize, and generate accurate and comprehensive answers remains limited. Further improvements would likely require more sophisticated retrieval strategies, better prompt engineering tailored to the retrieved context, and potentially fine-tuning the LLM on medical-specific data. The current results indicate that the retrieved context is not consistently leading to accurate or complete answers across various parameter settings.

### Actionable Insights and Recommendations

**Observations and Insights:**

* The base LLM struggles with medical accuracy without external knowledge.
* Prompt engineering can influence the style but not the core factual correctness.
* RAG provides relevant context but doesn't guarantee accurate or complete answers.
* Fine-tuning RAG parameters has a variable impact and requires careful experimentation.
* Initial RAG results showed issues with both groundedness and completeness.

**Recommendations for the Business:**

* **Knowledge Base:** Ensure the medical manual is comprehensive, up-to-date, and well-structured for effective chunking and retrieval. Consider adding metadata or categorizing information.
* **RAG System:**
    * Experiment with different chunking strategies and text splitters.
    * Explore more advanced retrieval methods beyond basic similarity search (e.g., MMR, self-querying).
    * Thoroughly evaluate different `k` values and potentially implement dynamic `k` based on query complexity.
    * Focus on prompt engineering techniques that instruct the LLM to explicitly cite the context and avoid making claims not supported by it.
* **LLM Usage:** Consider evaluating LLMs specifically fine-tuned for medical or scientific domains, as they may have a better baseline understanding.
* **Evaluation and Monitoring:** Implement a robust evaluation framework using both automated metrics (e.g., ROUGE, BLEU for answer quality; metrics for groundedness against the context) and human evaluation by medical professionals. Continuously monitor the system for accuracy and user feedback.
* **Overall Strategy:** A hybrid approach leveraging a well-maintained and comprehensive medical knowledge base with a carefully tuned RAG system is the most promising direction. Prompt engineering should be used to guide the LLM to be more grounded and specific. Initial deployment should focus on areas where high accuracy is critical, with rigorous testing and validation.

# Thank You